# Question 4: The West Coast Heatwave, Figure 1

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

from pathlib import Path

# Small style adjustments for more readable plots
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["font.size"] = 14

In [2]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import fsspec

import xarray as xr
xr.set_options(display_style="html")  # display dataset nicely

import warnings
warnings.simplefilter("ignore")  # filter some warning messages

# code features from https://scitools.org.uk/cartopy/docs/v0.14/matplotlib/feature_interface.html
crs = ccrs.PlateCarree()  # set projection

In [3]:
ds = xr.open_dataset("~/shared/climate-data/ds_hw.nc")

## Calculating SST Data from 2002-2012 for Anomaly Calculations

Figure 1 shows the monthly anomalies for 2014-2016, using SST from 2002-2012 per month and per location as the baseline. So, we first need to calculate this baseline, as well as gathering our relevant years.

In [4]:
trend_data = ds.analysed_sst.sel(time=slice('2002', '2012'))

baseline = trend_data.groupby('time.dayofyear').mean(skipna=True)

## Creating Figures

We now plot the anomalies for 2014, 2015, and 2016 (only through September for 2016) based on location. We calculate the anomaly per month by subtracting the baseline month mean from the observed month mean, and then plot these on the maps based on the longitude and latitude. 

In [12]:
def get_anomaly(year, month):
    '''Calculates the anomaly for the month in the given year year
    '''
    get_year = ds.analysed_sst.sel(time = (ds.time.dt.year == year))
    month_mean = get_year.sel(time = (get_year.time.dt.month == month)).mean(dim='time')
    return month_mean - baseline.sel(month == month)

In [13]:
def annotate_map(ax, month):
    '''
    Helper function for adding latitude/longitude ticks,
    land lines, and subplot letter label to map.
    '''
    # add lat/lon axis ticks
    ax.set_yticks([35, 40, 45], crs=crs)
    ax.set_ylabel(None)
    ax.set_xticks([-130, -120], crs=crs)
    ax.set_xlabel(None)

    # add land & coastlines
    ax.coastlines("10m", color="k")
    ax.add_feature(cfeature.LAND, color="grey")
    ax.add_feature(cfeature.STATES.with_scale("10m"))

    # add subplot month label in the top right
    ax.text(-118.5, 48, month, c='white', size='xx-large',
            weight=600, ha='right', va='top')

In [14]:
def make_fig(ax, month):
    '''Plots given anomaly
    '''
    # colorbar customization
    cbar_kwargs = {
        'location': 'bottom',
        'orientation': 'horizontal',
        'pad': 0.1,
        'aspect': 30,
        'label': '$\Delta$ SST ($\degree$C)',
        'ticks': np.arange(6),
    }

    anomaly.plot(ax=ax, transform=crs, cmap='jet',
                        cbar_kwargs=cbar_kwargs,
                        vmin=0, vmax=5
    )

    annotate_map(ax, month)

In [15]:
plt.figure()
row = 1
column = 1
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for year in np.arange(2014, 2017):
    for month in np.arange(1, 13):
        if year == 16 & month == 9: # don't need to plot beyond September for 2016
            break
        anomaly = get_anomaly(year, month)
        make_fig(plt.subplot(month, year - 2013, year - 2013), months[month-1])

ValueError: the first argument to .sel must be a dictionary

<Figure size 800x600 with 0 Axes>